In [1]:
import torch
import pandas as pd
import numpy as np

In [2]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")
test_np = test.values
train_np = train.values

In [3]:
trainX_np = train_np[:, :-1]  #rows, and last col
trainY_np = train_np[:, -1:] 

testX_np = test_np[:, :-1] 
testY_np = test_np[:, -1:]  

In [4]:
trainX = torch.tensor(trainX_np, dtype=torch.float64)
trainY = torch.tensor(trainY_np, dtype=torch.float64) 

testX = torch.tensor(testX_np, dtype=torch.float64)
testY = torch.tensor(testY_np, dtype=torch.float64)

In [5]:
#q1
q1 = trainX.nelement() * trainX.element_size()
q1

83520

In [6]:
#q2
trainX_16 = trainX.to(dtype=torch.float16)
trainX_64 = trainX_16.to(dtype=torch.float64)
diff = abs(trainX-trainX_64)
q2 = diff.max().item()
q2

0.0

In [7]:
#q3
q3 = torch.cuda.is_available()
q3

False

In [8]:
#q4
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)

q4 = (testX[0]@coef).item()
q4

9.844

In [9]:
#q5
q5=(testX@coef).mean().item()
q5

12.073632183908046

In [10]:
#q6
x = torch.tensor(0.0)
y = x ** 2 - 8 * x + 19
float(y)

19.0

In [11]:
#q7
def f(x):
    return x**2 - 8*x + 19
    
x =torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x],lr=0.01)

for epoch in range(500):
    y = f(x)
    y.backward()
    optimizer.step() 
    optimizer.zero_grad() 
    
x.item()

3.999835968017578

In [12]:
#q8
loss_fn = torch.nn.MSELoss()

coef = torch.zeros((trainX.shape[1], 1), dtype=torch.float64, requires_grad=True)
predictions = trainX @ coef

mse = loss_fn(predictions, trainY)
mse.item() 

197.8007662835249

In [13]:
#q9
torch.manual_seed(544)

ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size = 50, shuffle = True)

optimizer = torch.optim.SGD([coef], lr = 0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions = batchX @ coef
        loss = loss_fn(predictions, batchY)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
final_predictions = trainX @ coef #for training
final_mse=loss_fn(final_predictions, trainY)
final_mse.item() 

26.8113940147193

In [14]:
#q10
test_pred=testX @coef #for test data
test_mse = loss_fn(test_pred,testY).item()
test_mse

29.058546925485512